In [ ]:
# Cell 1: Setup & Data Preparation

# --- ติดตั้ง Libraries ---
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install opencv-python-headless pandas numpy matplotlib scikit-learn --quiet

# --- Import Libraries ---
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
from collections import defaultdict

# --- จำลองการสร้างข้อมูล (ในชีวิตจริง ส่วนนี้คือการเตรียมข้อมูลของคุณ) ---
# สมมติว่าคุณมีโฟลเดอร์ 'license_plates' ที่เก็บภาพป้ายทะเบียน
# และมีไฟล์ 'labels.csv' หน้าตาแบบนี้:
#
# filename,text
# plate001.png,2กข 1234
# plate002.png,1ขง 5678
# plate003.png,กท 9999

# สร้างโฟลเดอร์และไฟล์จำลอง
if not os.path.exists('license_plates'):
    os.makedirs('license_plates')

# สร้าง DataFrame จำลอง
mock_data = {
    'filename': ['plate001.png', 'plate002.png', 'plate003.png', 'plate004.png'],
    'text': ['2กข1234', '1ขง5678', 'กท9999', '9กอ777'] # เอาเว้นวรรคออกเพื่อความง่าย
}
df = pd.DataFrame(mock_data)

# สร้างไฟล์ภาพเปล่าๆ จำลอง
for fname in df['filename']:
    cv2.imwrite(os.path.join('license_plates', fname), np.zeros((50, 200, 3), dtype=np.uint8))

print("Data simulation complete.")
df.head()


In [ ]:
# Cell 2: Character Mapping

# --- กำหนดชุดตัวอักษรและตัวเลขทั้งหมดที่เป็นไปได้ ---
CHARS = 'กขคงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรลวศษสหฬอฮ0123456789'
# เพิ่ม CTC blank token ที่ index 0
CTC_CHARS = '-' + CHARS

# --- สร้าง Dictionary สำหรับแปลงตัวอักษรเป็นตัวเลข และ ngược lại ---
char_to_int = {char: i for i, char in enumerate(CTC_CHARS)}
int_to_char = {i: char for i, char in enumerate(CTC_CHARS)}

NUM_CLASSES = len(CTC_CHARS)

print(f"Total characters: {len(CHARS)}")
print(f"Total classes (including CTC blank): {NUM_CLASSES}")
print("Sample mapping:", {k: char_to_int[k] for k in list(char_to_int)[:5]})


In [ ]:
# Cell 3: Custom Dataset for OCR

class OCRDataset(Dataset):
    def __init__(self, df, img_dir, char_to_int_map):
        self.df = df
        self.img_dir = img_dir
        self.char_to_int = char_to_int_map
        # ปรับขนาดภาพและแปลงเป็น Tensor
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((64, 200)), # (Height, Width)
            transforms.Grayscale(num_output_channels=1), # ใช้ภาพขาว-ดำ
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = cv2.imread(img_path)
        image = self.transform(image)
        
        text_label = row['text']
        encoded_label = [self.char_to_int[char] for char in text_label]
        
        return {
            "image": image,
            "label": torch.tensor(encoded_label, dtype=torch.long),
            "label_length": torch.tensor([len(text_label)], dtype=torch.long)
        }
        
# --- สร้าง DataLoader ---
# CTC Loss ต้องการ Batch ที่ข้อมูลถูกเรียงต่อกัน เราจึงต้องสร้าง custom collate_fn
def collate_fn(batch):
    images = torch.stack([item['image'] for item in batch])
    labels = [item['label'] for item in batch]
    label_lengths = torch.stack([item['label_length'] for item in batch]).squeeze()

    # Padding labels to the same length
    labels_padded = nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=0)
    
    return images, labels_padded, label_lengths


# สร้าง Dataset
full_dataset = OCRDataset(df, 'license_plates', char_to_int)
# สร้าง DataLoader
data_loader = DataLoader(
    dataset=full_dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=collate_fn
)

# --- ตรวจสอบข้อมูลจาก DataLoader ---
images, labels, label_lengths = next(iter(data_loader))
print("Image batch shape:", images.shape) # (Batch, Channels, Height, Width)
print("Labels batch shape:", labels.shape)
print("Label lengths:", label_lengths)



In [ ]:
# Cell 4: CRNN Model Architecture

class CRNN(nn.Module):
    def __init__(self, num_classes):
        super(CRNN, self).__init__()
        
        # --- CNN Feature Extractor ---
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool2d(2, 2), # -> 32x100
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool2d(2, 2), # -> 16x50
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool2d((2, 1), (2, 1)), # -> 8x50
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU(True),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, kernel_size=3, padding=1), nn.ReLU(True),
            nn.MaxPool2d((2, 1), (2, 1)), # -> 4x50
            nn.Conv2d(256, 512, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(512), nn.ReLU(True) # -> 4x50
        )
        
        # --- Reshape for RNN ---
        # หลัง CNN, shape จะเป็น (batch, 512, 4, 50)
        # เราต้องแปลงเป็น (batch, 4*512, 50) -> (batch, 2048, 50)
        # แล้วสลับมิติเป็น (seq_len, batch, input_size) สำหรับ RNN
        # seq_len คือความกว้างของภาพ (50), input_size คือ 2048
        
        # --- RNN Sequence Predictor ---
        self.rnn = nn.Sequential(
            nn.LSTM(2048, 256, bidirectional=True),
            nn.LSTM(512, 256, bidirectional=True)
        )
        
        # --- Fully Connected Layer ---
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        # CNN part
        x = self.cnn(x)
        
        # Reshape for RNN
        b, c, h, w = x.size()
        x = x.view(b, c * h, w)
        x = x.permute(2, 0, 1) # (Width, Batch, Channels*Height)
        
        # RNN part
        x, _ = self.rnn(x)
        
        # FC part
        x = self.fc(x) # Output shape: (seq_len, batch, num_classes)
        
        return x

# --- สร้างโมเดลและย้ายไป GPU ถ้ามี ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = CRNN(num_classes=NUM_CLASSES).to(DEVICE)
print(f"Model created and moved to {DEVICE}.")


In [ ]:
# Cell 5 (Updated): Training Loop with CTC Loss and Model Saving

# --- กำหนด Loss และ Optimizer ---
criterion = nn.CTCLoss(blank=0, zero_infinity=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 50 
MODEL_SAVE_PATH = "crnn_tha_license_plate.pth"

model.train()
import tqdm
print("Starting training...")
for epoch in range(EPOCHS):
    epoch_loss = 0
    # ใช้ tqdm เพื่อให้เห็น progress bar
    for batch in tqdm(data_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        images = batch["image"].to(DEVICE)
        labels = batch["label"]
        label_lengths = batch["label_length"]
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        log_probs = nn.functional.log_softmax(outputs, dim=2)
        
        # Calculate loss
        input_lengths = torch.full(size=(images.size(0),), fill_value=outputs.size(0), dtype=torch.long)
        loss = criterion(log_probs, labels, input_lengths, label_lengths)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
    print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {epoch_loss/len(data_loader):.4f}")

# --- บันทึกโมเดลหลังจากการฝึกเสร็จสิ้น ---
torch.save(model.state_dict(), MODEL_SAVE_PATH)
print(f"\nTraining finished. Model saved to {MODEL_SAVE_PATH}")


In [ ]:
# Cell 6: Inference and Decoding

def decode_prediction(preds, int_to_char_map):
    preds = preds.permute(1, 0, 2) # (Batch, SeqLen, Classes)
    preds = torch.argmax(preds, dim=2)
    preds = preds.detach().cpu().numpy()
    
    decoded_texts = []
    for pred in preds:
        text = ""
        for i in range(len(pred)):
            # ลบตัวซ้ำและ blank
            if pred[i] != 0 and (i == 0 or pred[i] != pred[i-1]):
                text += int_to_char_map[pred[i]]
        decoded_texts.append(text)
    return decoded_texts


# --- ทดลองทำนายผล ---
model.eval()
with torch.no_grad():
    # นำข้อมูลชุดแรกมาทดสอบ
    images, labels, label_lengths = next(iter(data_loader))
    images = images.to(DEVICE)
    
    # ทำนายผล
    preds = model(images)
    
    # ถอดรหัส
    predicted_texts = decode_prediction(preds, int_to_char)
    
    # ถอดรหัส Ground Truth เพื่อเปรียบเทียบ
    true_texts = []
    for label in labels:
        true_texts.append("".join([int_to_char[i] for i in label if i != 0]))
        
    # แสดงผล
    for i in range(len(images)):
        plt.imshow(images[i].cpu().squeeze(), cmap='gray')
        plt.title(f"True: {true_texts[i]}\nPred: {predicted_texts[i]}")
        plt.axis('off')
        plt.show()



In [ ]:
# Cell 7: Inference on Test Set and Submission File Generation

# --- 1. จำลอง Test Set ---
# ในชีวิตจริง คุณจะมีโฟลเดอร์ test อยู่แล้ว
TEST_IMG_DIR = 'test_plates'
if not os.path.exists(TEST_IMG_DIR):
    os.makedirs(TEST_IMG_DIR)

# สร้าง DataFrame สำหรับ test set
mock_test_data = {'filename': ['test001.png', 'test002.png']}
test_df = pd.DataFrame(mock_test_data)

# สร้างไฟล์ภาพเปล่าๆ จำลอง
for fname in test_df['filename']:
    cv2.imwrite(os.path.join(TEST_IMG_DIR, fname), np.zeros((50, 200, 3), dtype=np.uint8))

print(f"Test set simulated with {len(test_df)} images.")


# --- 2. สร้าง Dataset และ DataLoader สำหรับ Test ---
class TestOCRDataset(Dataset):
    def __init__(self, df, img_dir):
        self.df = df
        self.img_dir = img_dir
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((64, 200)),
            transforms.Grayscale(num_output_channels=1),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        filename = row['filename']
        img_path = os.path.join(self.img_dir, filename)
        image = cv2.imread(img_path)
        image = self.transform(image)
        return image, filename

test_dataset = TestOCRDataset(test_df, TEST_IMG_DIR)
# สำหรับ Test, batch_size อาจจะใหญ่ขึ้นได้ และไม่ต้องมี collate_fn ที่ซับซ้อน
test_loader = DataLoader(dataset=test_dataset, batch_size=4, shuffle=False)


# --- 3. โหลดโมเดลและทำนายผล ---
# สร้าง instance ของโมเดลและโหลด state ที่บันทึกไว้
inference_model = CRNN(num_classes=NUM_CLASSES).to(DEVICE)
inference_model.load_state_dict(torch.load(MODEL_SAVE_PATH))
inference_model.eval() # ตั้งเป็น evaluation mode

# ฟังก์ชันถอดรหัสจากขั้นตอนที่ 6
def decode_prediction(preds, int_to_char_map):
    preds = preds.permute(1, 0, 2)
    preds = torch.argmax(preds, dim=2)
    preds = preds.detach().cpu().numpy()
    
    decoded_texts = []
    for pred in preds:
        text = ""
        for i in range(len(pred)):
            if pred[i] != 0 and (i == 0 or pred[i] != pred[i-1]):
                text += int_to_char_map[pred[i]]
        decoded_texts.append(text)
    return decoded_texts

# เก็บผลลัพธ์
all_filenames = []
all_predictions = []

with torch.no_grad():
    for images, filenames in tqdm(test_loader, desc="Generating Predictions"):
        images = images.to(DEVICE)
        
        # ทำนายผล
        preds = inference_model(images)
        
        # ถอดรหัส
        predicted_texts = decode_prediction(preds, int_to_char)
        
        all_filenames.extend(filenames)
        all_predictions.extend(predicted_texts)


# --- 4. สร้างไฟล์ Submission ---
submission_df = pd.DataFrame({
    'filename': all_filenames,
    'predicted_text': all_predictions
})

submission_df.to_csv('submission.csv', index=False)

print("\nsubmission.csv created successfully!")
display(submission_df.head())
